In [122]:
import nba_api
from nba_api.stats.endpoints import shotchartdetail
import pandas as pd
import json
import requests
import datetime
import numpy as np
pd.set_option('display.max_columns',None)

## Regular Season API Connection

In [4]:
#Function to connect to NBA shot chart detail API endpoint
def create_combined_dataframe(seasons):
    dataframes = []
    
    for season in seasons:
        # Create JSON request for the season
        shot_json = shotchartdetail.ShotChartDetail(
            team_id=0,
            player_id=0,
            context_measure_simple='FGA',
            season_nullable=season,
            season_type_all_star='Regular Season'
        )
        
        # Load data into a Python dictionary
        shot_data = json.loads(shot_json.get_json())
        
        # Get the relevant data from the dictionary
        relevant_data = shot_data['resultSets'][0]
        
        # Get the headers and row data
        headers = relevant_data['headers']
        rows = relevant_data['rowSet']
        
        # Create pandas DataFrame
        nba_shot_data = pd.DataFrame(rows, columns=headers)
        
        dataframes.append(nba_shot_data)
    
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    return combined_df

In [8]:
#Breaking all seasons of interest into lists to feed into our function - running all the seasons together caused time out errors
seasons_1 = ['2022-23', '2021-22', '2020-21', '2019-20', '2018-19'] 
combined_dataframe_1 = create_combined_dataframe(seasons_1)

In [9]:
seasons_2 = ['2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12']
combined_dataframe_2 = create_combined_dataframe(seasons_2)

In [10]:
seasons_3 = ['2010-11', '2009-10', '2008-09', '2007-08', '2006-07', '2005-06']
combined_dataframe_3 = create_combined_dataframe(seasons_3)

In [15]:
seasons_4 = ['2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00'] 
combined_dataframe_4 = create_combined_dataframe(seasons_4)

In [16]:
seasons_5 = ['1998-99', '1997-98', '1996-97', '1995-96', '1994-95', '1993-94']
combined_dataframe_5 = create_combined_dataframe(seasons_5)

In [17]:
seasons_6 = ['1992-93', '1991-92', '1990-91', '1989-90']
combined_dataframe_6 = create_combined_dataframe(seasons_6)

In [108]:
#Creating our dataframe with all shot data from 1990 - 2023
nba_shot_data_consolidated = pd.concat([combined_dataframe_1, combined_dataframe_2, combined_dataframe_3, combined_dataframe_4, combined_dataframe_5, combined_dataframe_6])

In [109]:
#Fix abbreviations for old teams to match abbreviations for today's teams
nba_shot_data_consolidated['HTM'] = nba_shot_data_consolidated['HTM'].replace({'NJN':'BKN','CHA':'CHH','VAN':'MEM','NOH':'NOP','NOK':'NOP','SEA':'OKC'})
nba_shot_data_consolidated['VTM'] = nba_shot_data_consolidated['VTM'].replace({'NJN':'BKN','CHA':'CHH','VAN':'MEM','NOH':'NOP','NOK':'NOP','SEA':'OKC'})

In [110]:
#Mapping out full team names to standardized abbreviations
final_team_abbreviations = {
'Atlanta Hawks':'ATL', 
'Boston Celtics':'BOS', 
'Brooklyn Nets':'BKN', 
'Charlotte Bobcats':'CHH',  
'Charlotte Hornets':'CHH', 
'Chicago Bulls':'CHI',
'Cleveland Cavaliers':'CLE', 
'Dallas Mavericks':'DAL', 
'Denver Nuggets':'DEN',
'Detroit Pistons':'DET', 
'Golden State Warriors':'GSW', 
'Houston Rockets':'HOU',
'Indiana Pacers':'IND', 
'LA Clippers':'LAC', 
'Los Angeles Clippers':'LAC', 
'Los Angeles Lakers':'LAL', 
'Memphis Grizzlies':'MEM',  
'Miami Heat':'MIA',
'Milwaukee Bucks':'MIL', 
'Minnesota Timberwolves':'MIN', 
'New Jersey Nets':'BKN', 
'New Orleans Hornets':'NOP',  
'New Orleans Pelicans':'NOP', 
'New Orleans/Oklahoma City Hornets':'NOP',
'New York Knicks':'NYK',
'Oklahoma City Thunder':'OKC', 
'Orlando Magic':'ORL', 
'Philadelphia 76ers':'PHI',
'Phoenix Suns':'PHX', 
'Portland Trail Blazers':'POR', 
'Sacramento Kings':'SAC',
'San Antonio Spurs':'SAS', 
'Seattle SuperSonics':'OKC', 
'Toronto Raptors':'TOR',
'Utah Jazz':'UTA', 
'Vancouver Grizzlies':'MEM', 
'Washington Bullets':'WAS',  
'Washington Wizards':'WAS'
}

#Regular season column
nba_shot_data_consolidated['TIME_OF_SEASON'] = 'Regular Season'

#Applying the above dictionary to create a new column, TEAM_NAME_NEW, that includes the new abbreviated names to match the HTM and VTM columns from the API
nba_shot_data_consolidated['TEAM_NAME_NEW'] = nba_shot_data_consolidated['TEAM_NAME'].map(final_team_abbreviations)

#New column to determine who the opposing team was
nba_shot_data_consolidated['OPPONENT'] = np.where(nba_shot_data_consolidated['TEAM_NAME_NEW'] == nba_shot_data_consolidated['HTM'], nba_shot_data_consolidated['VTM'], nba_shot_data_consolidated['HTM'])

#New column to determine who the home/away team was for each player
nba_shot_data_consolidated['HOME_AWAY'] = np.where(nba_shot_data_consolidated['TEAM_NAME_NEW'] == nba_shot_data_consolidated['HTM'], 'Home', 'Away')

#New column fixing date format
nba_shot_data_consolidated['GAME_DATE_FIXED'] = pd.to_datetime(nba_shot_data_consolidated['GAME_DATE'], format='%Y%m%d').dt.strftime('%m-%d-%Y')
nba_shot_data_consolidated['GAME_DATE_FIXED'] = pd.to_datetime(nba_shot_data_consolidated['GAME_DATE_FIXED'])

seasons_dict = {
    (pd.to_datetime('10-01-1989'), pd.to_datetime('09-30-1990')): '1989-1990',
    (pd.to_datetime('10-01-1990'), pd.to_datetime('09-30-1991')): '1990-1991',
    (pd.to_datetime('10-01-1991'), pd.to_datetime('09-30-1992')): '1991-1992',
    (pd.to_datetime('10-01-1992'), pd.to_datetime('09-30-1993')): '1992-1993',
    (pd.to_datetime('10-01-1993'), pd.to_datetime('09-30-1994')): '1993-1994',
    (pd.to_datetime('10-01-1994'), pd.to_datetime('09-30-1995')): '1994-1995',
    (pd.to_datetime('10-01-1995'), pd.to_datetime('09-30-1996')): '1995-1996',
    (pd.to_datetime('10-01-1996'), pd.to_datetime('09-30-1997')): '1996-1997',
    (pd.to_datetime('10-01-1997'), pd.to_datetime('09-30-1998')): '1997-1998',
    (pd.to_datetime('10-01-1998'), pd.to_datetime('09-30-1999')): '1998-1999',
    (pd.to_datetime('10-01-1999'), pd.to_datetime('09-30-2000')): '1999-2000',
    (pd.to_datetime('10-01-2000'), pd.to_datetime('09-30-2001')): '2000-2001',
    (pd.to_datetime('10-01-2001'), pd.to_datetime('09-30-2002')): '2001-2002',
    (pd.to_datetime('10-01-2002'), pd.to_datetime('09-30-2003')): '2002-2003',
    (pd.to_datetime('10-01-2003'), pd.to_datetime('09-30-2004')): '2003-2004',
    (pd.to_datetime('10-01-2004'), pd.to_datetime('09-30-2005')): '2004-2005',
    (pd.to_datetime('10-01-2005'), pd.to_datetime('09-30-2006')): '2005-2006',
    (pd.to_datetime('10-01-2006'), pd.to_datetime('09-30-2007')): '2006-2007',
    (pd.to_datetime('10-01-2007'), pd.to_datetime('09-30-2008')): '2007-2008',
    (pd.to_datetime('10-01-2008'), pd.to_datetime('09-30-2009')): '2008-2009',
    (pd.to_datetime('10-01-2009'), pd.to_datetime('09-30-2010')): '2009-2010',
    (pd.to_datetime('10-01-2010'), pd.to_datetime('09-30-2011')): '2010-2011',
    (pd.to_datetime('10-01-2011'), pd.to_datetime('09-30-2012')): '2011-2012',
    (pd.to_datetime('10-01-2012'), pd.to_datetime('09-30-2013')): '2012-2013',
    (pd.to_datetime('10-01-2013'), pd.to_datetime('09-30-2014')): '2013-2014',
    (pd.to_datetime('10-01-2014'), pd.to_datetime('09-30-2015')): '2014-2015',
    (pd.to_datetime('10-01-2015'), pd.to_datetime('09-30-2016')): '2015-2016',
    (pd.to_datetime('10-01-2016'), pd.to_datetime('09-30-2017')): '2016-2017',
    (pd.to_datetime('10-01-2017'), pd.to_datetime('09-30-2018')): '2017-2018',
    (pd.to_datetime('10-01-2018'), pd.to_datetime('09-30-2019')): '2018-2019',
    (pd.to_datetime('10-01-2019'), pd.to_datetime('09-30-2020')): '2019-2020',
    (pd.to_datetime('10-01-2020'), pd.to_datetime('09-30-2021')): '2020-2021',
    (pd.to_datetime('10-01-2021'), pd.to_datetime('09-30-2022')): '2021-2022',
    (pd.to_datetime('10-01-2022'), pd.to_datetime('09-30-2023')): '2022-2023'
}


#Function to map date ranges to seasons
def map_season(date):
    for interval, season in seasons_dict.items():
        if interval[0] <= date <= interval[1]:
            return season
    return None

#Apply the map_season function to create the 'SEASON' column
nba_shot_data_consolidated['SEASON'] = nba_shot_data_consolidated['GAME_DATE_FIXED'].apply(map_season)

#Combining minutes and seconds columns
nba_shot_data_consolidated['TIME_REMAINING'] = nba_shot_data_consolidated['MINUTES_REMAINING'].astype(str).str.zfill(2) + ':' + nba_shot_data_consolidated['SECONDS_REMAINING'].astype(str).str.zfill(2)

#Total seconds remaining in quarter
nba_shot_data_consolidated['SECONDS_REMAINING_QUARTER'] = (nba_shot_data_consolidated['MINUTES_REMAINING']*60) + nba_shot_data_consolidated['SECONDS_REMAINING']

nba_shot_data_consolidated.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,TIME_OF_SEASON,TEAM_NAME_NEW,OPPONENT,HOME_AWAY,GAME_DATE_FIXED,SEASON,TIME_REMAINING,SECONDS_REMAINING_QUARTER
0,Shot Chart Detail,0022200001,7,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,11,38,Missed Shot,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,12,-118,50,1,0,20221018,BOS,PHI,Regular Season,PHI,BOS,Away,2022-10-18,2022-2023,11:38,698
1,Shot Chart Detail,0022200001,11,203935,Marcus Smart,1610612738,Boston Celtics,1,11,15,Made Shot,Driving Floating Bank Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),8-16 ft.,13,120,55,1,1,20221018,BOS,PHI,Regular Season,BOS,PHI,Home,2022-10-18,2022-2023,11:15,675
2,Shot Chart Detail,0022200001,12,202699,Tobias Harris,1610612755,Philadelphia 76ers,1,11,5,Missed Shot,Driving Floating Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,50,135,1,0,20221018,BOS,PHI,Regular Season,PHI,BOS,Away,2022-10-18,2022-2023,11:05,665
3,Shot Chart Detail,0022200001,14,202699,Tobias Harris,1610612755,Philadelphia 76ers,1,11,3,Made Shot,Tip Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,1,1,20221018,BOS,PHI,Regular Season,PHI,BOS,Away,2022-10-18,2022-2023,11:03,663
4,Shot Chart Detail,0022200001,15,1628369,Jayson Tatum,1610612738,Boston Celtics,1,10,46,Made Shot,Jump Shot,3PT Field Goal,Left Corner 3,Left Side(L),24+ ft.,23,-232,49,1,1,20221018,BOS,PHI,Regular Season,BOS,PHI,Home,2022-10-18,2022-2023,10:46,646


## Playoffs API Connection

In [85]:
#Function to connect to NBA shot chart detail API endpoint
def create_combined_dataframe_playoffs(seasons):
    dataframes = []
    
    for season in seasons:
        # Create JSON request for the season
        shot_json = shotchartdetail.ShotChartDetail(
            team_id=0,
            player_id=0,
            context_measure_simple='FGA',
            season_nullable=season,
            season_type_all_star='Playoffs' #change to Playoffs here
        )
        
        # Load data into a Python dictionary
        shot_data = json.loads(shot_json.get_json())
        
        # Get the relevant data from the dictionary
        relevant_data = shot_data['resultSets'][0]
        
        # Get the headers and row data
        headers = relevant_data['headers']
        rows = relevant_data['rowSet']
        
        # Create pandas DataFrame
        nba_shot_data = pd.DataFrame(rows, columns=headers)
        
        dataframes.append(nba_shot_data)
    
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    return combined_df

In [86]:
#Breaking all seasons of interest into lists to feed into our function - running all the seasons together caused time out errors
seasons_1_playoffs= ['2022-23', '2021-22', '2020-21', '2019-20', '2018-19'] 
combined_dataframe_1_playoffs = create_combined_dataframe_playoffs(seasons_1_playoffs)

In [87]:
seasons_2_playoffs = ['2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12']
combined_dataframe_2_playoffs = create_combined_dataframe_playoffs(seasons_2_playoffs)

In [88]:
seasons_3_playoffs = ['2010-11', '2009-10', '2008-09', '2007-08', '2006-07', '2005-06']
combined_dataframe_3_playoffs = create_combined_dataframe_playoffs(seasons_3_playoffs)

In [89]:
seasons_4_playoffs = ['2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00'] 
combined_dataframe_4_playoffs = create_combined_dataframe_playoffs(seasons_4_playoffs)

In [90]:
seasons_5_playoffs = ['1998-99', '1997-98', '1996-97', '1995-96', '1994-95', '1993-94']
combined_dataframe_5_playoffs = create_combined_dataframe_playoffs(seasons_5_playoffs)

In [91]:
seasons_6_playoffs = ['1992-93', '1991-92', '1990-91', '1989-90']
combined_dataframe_6_playoffs = create_combined_dataframe_playoffs(seasons_6_playoffs)

In [105]:
#Creating our dataframe with all shot data from 1990 - 2023
nba_shot_data_consolidated_playoffs = pd.concat([combined_dataframe_1_playoffs, combined_dataframe_2_playoffs, combined_dataframe_3_playoffs, combined_dataframe_4_playoffs, combined_dataframe_5_playoffs, combined_dataframe_6_playoffs])

In [106]:
#Fix abbreviations for old teams to match abbreviations for today's teams
nba_shot_data_consolidated_playoffs['HTM'] = nba_shot_data_consolidated_playoffs['HTM'].replace({'NJN':'BKN','CHA':'CHH','VAN':'MEM','NOH':'NOP','NOK':'NOP','SEA':'OKC'})
nba_shot_data_consolidated_playoffs['VTM'] = nba_shot_data_consolidated_playoffs['VTM'].replace({'NJN':'BKN','CHA':'CHH','VAN':'MEM','NOH':'NOP','NOK':'NOP','SEA':'OKC'})

In [107]:
#Mapping out full team names to standardized abbreviations
final_team_abbreviations = {
'Atlanta Hawks':'ATL', 
'Boston Celtics':'BOS', 
'Brooklyn Nets':'BKN', 
'Charlotte Bobcats':'CHH',  
'Charlotte Hornets':'CHH', 
'Chicago Bulls':'CHI',
'Cleveland Cavaliers':'CLE', 
'Dallas Mavericks':'DAL', 
'Denver Nuggets':'DEN',
'Detroit Pistons':'DET', 
'Golden State Warriors':'GSW', 
'Houston Rockets':'HOU',
'Indiana Pacers':'IND', 
'LA Clippers':'LAC', 
'Los Angeles Clippers':'LAC', 
'Los Angeles Lakers':'LAL', 
'Memphis Grizzlies':'MEM',  
'Miami Heat':'MIA',
'Milwaukee Bucks':'MIL', 
'Minnesota Timberwolves':'MIN', 
'New Jersey Nets':'BKN', 
'New Orleans Hornets':'NOP',  
'New Orleans Pelicans':'NOP', 
'New Orleans/Oklahoma City Hornets':'NOP',
'New York Knicks':'NYK',
'Oklahoma City Thunder':'OKC', 
'Orlando Magic':'ORL', 
'Philadelphia 76ers':'PHI',
'Phoenix Suns':'PHX', 
'Portland Trail Blazers':'POR', 
'Sacramento Kings':'SAC',
'San Antonio Spurs':'SAS', 
'Seattle SuperSonics':'OKC', 
'Toronto Raptors':'TOR',
'Utah Jazz':'UTA', 
'Vancouver Grizzlies':'MEM', 
'Washington Bullets':'WAS',  
'Washington Wizards':'WAS'
}

#Regular season column
nba_shot_data_consolidated_playoffs['TIME_OF_SEASON'] = 'Playoffs'

#Applying the above dictionary to create a new column, TEAM_NAME_NEW, that includes the new abbreviated names to match the HTM and VTM columns from the API
nba_shot_data_consolidated_playoffs['TEAM_NAME_NEW'] = nba_shot_data_consolidated_playoffs['TEAM_NAME'].map(final_team_abbreviations)

#New column to determine who the opposing team was
nba_shot_data_consolidated_playoffs['OPPONENT'] = np.where(nba_shot_data_consolidated_playoffs['TEAM_NAME_NEW'] == nba_shot_data_consolidated_playoffs['HTM'], nba_shot_data_consolidated_playoffs['VTM'], nba_shot_data_consolidated_playoffs['HTM'])

#New column to determine who the home/away team was for each player
nba_shot_data_consolidated_playoffs['HOME_AWAY'] = np.where(nba_shot_data_consolidated_playoffs['TEAM_NAME_NEW'] == nba_shot_data_consolidated_playoffs['HTM'], 'Home', 'Away')

#New column fixing date format
nba_shot_data_consolidated_playoffs['GAME_DATE_FIXED'] = pd.to_datetime(nba_shot_data_consolidated_playoffs['GAME_DATE'], format='%Y%m%d').dt.strftime('%m-%d-%Y')
nba_shot_data_consolidated_playoffs['GAME_DATE_FIXED'] = pd.to_datetime(nba_shot_data_consolidated_playoffs['GAME_DATE_FIXED'])

seasons_dict = {
    (pd.to_datetime('10-01-1989'), pd.to_datetime('09-30-1990')): '1989-1990',
    (pd.to_datetime('10-01-1990'), pd.to_datetime('09-30-1991')): '1990-1991',
    (pd.to_datetime('10-01-1991'), pd.to_datetime('09-30-1992')): '1991-1992',
    (pd.to_datetime('10-01-1992'), pd.to_datetime('09-30-1993')): '1992-1993',
    (pd.to_datetime('10-01-1993'), pd.to_datetime('09-30-1994')): '1993-1994',
    (pd.to_datetime('10-01-1994'), pd.to_datetime('09-30-1995')): '1994-1995',
    (pd.to_datetime('10-01-1995'), pd.to_datetime('09-30-1996')): '1995-1996',
    (pd.to_datetime('10-01-1996'), pd.to_datetime('09-30-1997')): '1996-1997',
    (pd.to_datetime('10-01-1997'), pd.to_datetime('09-30-1998')): '1997-1998',
    (pd.to_datetime('10-01-1998'), pd.to_datetime('09-30-1999')): '1998-1999',
    (pd.to_datetime('10-01-1999'), pd.to_datetime('09-30-2000')): '1999-2000',
    (pd.to_datetime('10-01-2000'), pd.to_datetime('09-30-2001')): '2000-2001',
    (pd.to_datetime('10-01-2001'), pd.to_datetime('09-30-2002')): '2001-2002',
    (pd.to_datetime('10-01-2002'), pd.to_datetime('09-30-2003')): '2002-2003',
    (pd.to_datetime('10-01-2003'), pd.to_datetime('09-30-2004')): '2003-2004',
    (pd.to_datetime('10-01-2004'), pd.to_datetime('09-30-2005')): '2004-2005',
    (pd.to_datetime('10-01-2005'), pd.to_datetime('09-30-2006')): '2005-2006',
    (pd.to_datetime('10-01-2006'), pd.to_datetime('09-30-2007')): '2006-2007',
    (pd.to_datetime('10-01-2007'), pd.to_datetime('09-30-2008')): '2007-2008',
    (pd.to_datetime('10-01-2008'), pd.to_datetime('09-30-2009')): '2008-2009',
    (pd.to_datetime('10-01-2009'), pd.to_datetime('09-30-2010')): '2009-2010',
    (pd.to_datetime('10-01-2010'), pd.to_datetime('09-30-2011')): '2010-2011',
    (pd.to_datetime('10-01-2011'), pd.to_datetime('09-30-2012')): '2011-2012',
    (pd.to_datetime('10-01-2012'), pd.to_datetime('09-30-2013')): '2012-2013',
    (pd.to_datetime('10-01-2013'), pd.to_datetime('09-30-2014')): '2013-2014',
    (pd.to_datetime('10-01-2014'), pd.to_datetime('09-30-2015')): '2014-2015',
    (pd.to_datetime('10-01-2015'), pd.to_datetime('09-30-2016')): '2015-2016',
    (pd.to_datetime('10-01-2016'), pd.to_datetime('09-30-2017')): '2016-2017',
    (pd.to_datetime('10-01-2017'), pd.to_datetime('09-30-2018')): '2017-2018',
    (pd.to_datetime('10-01-2018'), pd.to_datetime('09-30-2019')): '2018-2019',
    (pd.to_datetime('10-01-2019'), pd.to_datetime('09-30-2020')): '2019-2020',
    (pd.to_datetime('10-01-2020'), pd.to_datetime('09-30-2021')): '2020-2021',
    (pd.to_datetime('10-01-2021'), pd.to_datetime('09-30-2022')): '2021-2022',
    (pd.to_datetime('10-01-2022'), pd.to_datetime('09-30-2023')): '2022-2023'
}


#Function to map date ranges to seasons
def map_season(date):
    for interval, season in seasons_dict.items():
        if interval[0] <= date <= interval[1]:
            return season
    return None

#Apply the map_season function to create the 'SEASON' column
nba_shot_data_consolidated_playoffs['SEASON'] = nba_shot_data_consolidated_playoffs['GAME_DATE_FIXED'].apply(map_season)

#Combining minutes and seconds columns
nba_shot_data_consolidated_playoffs['TIME_REMAINING'] = nba_shot_data_consolidated_playoffs['MINUTES_REMAINING'].astype(str).str.zfill(2) + ':' + nba_shot_data_consolidated_playoffs['SECONDS_REMAINING'].astype(str).str.zfill(2)

#Total seconds remaining in quarter
nba_shot_data_consolidated_playoffs['SECONDS_REMAINING_QUARTER'] = (nba_shot_data_consolidated_playoffs['MINUTES_REMAINING']*60) + nba_shot_data_consolidated_playoffs['SECONDS_REMAINING']

nba_shot_data_consolidated_playoffs.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,TIME_OF_SEASON,TEAM_NAME_NEW,OPPONENT,HOME_AWAY,GAME_DATE_FIXED,SEASON,TIME_REMAINING,SECONDS_REMAINING_QUARTER
0,Shot Chart Detail,0042200101,7,1628389,Bam Adebayo,1610612748,Miami Heat,1,11,36,Missed Shot,Pullup Jump shot,2PT Field Goal,Mid-Range,Center(C),8-16 ft.,13,11,139,1,0,20230416,MIL,MIA,Playoffs,MIA,MIL,Away,2023-04-16,2022-2023,11:36,696
1,Shot Chart Detail,0042200101,10,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,11,23,Made Shot,Driving Finger Roll Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,1,22,1,1,20230416,MIL,MIA,Playoffs,MIL,MIA,Home,2023-04-16,2022-2023,11:23,683
2,Shot Chart Detail,0042200101,12,1629639,Tyler Herro,1610612748,Miami Heat,1,11,13,Made Shot,Driving Floating Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,12,-51,118,1,1,20230416,MIL,MIA,Playoffs,MIA,MIL,Away,2023-04-16,2022-2023,11:13,673
3,Shot Chart Detail,0042200101,14,1628960,Grayson Allen,1610612749,Milwaukee Bucks,1,11,1,Made Shot,Alley Oop Dunk Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,0,26,1,1,20230416,MIL,MIA,Playoffs,MIL,MIA,Home,2023-04-16,2022-2023,11:01,661
4,Shot Chart Detail,0042200101,16,1629639,Tyler Herro,1610612748,Miami Heat,1,10,53,Made Shot,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,25,55,253,1,1,20230416,MIL,MIA,Playoffs,MIA,MIL,Away,2023-04-16,2022-2023,10:53,653


## Combine regular season and playoff data then export to CSV

In [119]:
nba_shot_data_final = pd.concat([nba_shot_data_consolidated, nba_shot_data_consolidated_playoffs])

In [113]:
nba_shot_data_final.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,TIME_OF_SEASON,TEAM_NAME_NEW,OPPONENT,HOME_AWAY,GAME_DATE_FIXED,SEASON,TIME_REMAINING,SECONDS_REMAINING_QUARTER
0,Shot Chart Detail,0022200001,7,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,11,38,Missed Shot,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,12,-118,50,1,0,20221018,BOS,PHI,Regular Season,PHI,BOS,Away,2022-10-18,2022-2023,11:38,698
1,Shot Chart Detail,0022200001,11,203935,Marcus Smart,1610612738,Boston Celtics,1,11,15,Made Shot,Driving Floating Bank Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),8-16 ft.,13,120,55,1,1,20221018,BOS,PHI,Regular Season,BOS,PHI,Home,2022-10-18,2022-2023,11:15,675
2,Shot Chart Detail,0022200001,12,202699,Tobias Harris,1610612755,Philadelphia 76ers,1,11,5,Missed Shot,Driving Floating Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,50,135,1,0,20221018,BOS,PHI,Regular Season,PHI,BOS,Away,2022-10-18,2022-2023,11:05,665
3,Shot Chart Detail,0022200001,14,202699,Tobias Harris,1610612755,Philadelphia 76ers,1,11,3,Made Shot,Tip Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,1,1,20221018,BOS,PHI,Regular Season,PHI,BOS,Away,2022-10-18,2022-2023,11:03,663
4,Shot Chart Detail,0022200001,15,1628369,Jayson Tatum,1610612738,Boston Celtics,1,10,46,Made Shot,Jump Shot,3PT Field Goal,Left Corner 3,Left Side(L),24+ ft.,23,-232,49,1,1,20221018,BOS,PHI,Regular Season,BOS,PHI,Home,2022-10-18,2022-2023,10:46,646


In [120]:
nba_shot_data_final.drop(columns=['GRID_TYPE','PLAYER_ID','TEAM_ID','HTM','VTM','GAME_DATE','TEAM_NAME','SHOT_ZONE_RANGE'],axis=1,inplace=True)
nba_shot_data_final.head()

,GAME_ID,GAME_EVENT_ID,PLAYER_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,TIME_OF_SEASON,TEAM_NAME_NEW,OPPONENT,HOME_AWAY,GAME_DATE_FIXED,SEASON,TIME_REMAINING,SECONDS_REMAINING_QUARTER
0,0022200001,7,Joel Embiid,1,11,38,Missed Shot,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Left Side(L),12,-118,50,1,0,Regular Season,PHI,BOS,Away,2022-10-18,2022-2023,11:38,698
1,0022200001,11,Marcus Smart,1,11,15,Made Shot,Driving Floating Bank Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),13,120,55,1,1,Regular Season,BOS,PHI,Home,2022-10-18,2022-2023,11:15,675
2,0022200001,12,Tobias Harris,1,11,5,Missed Shot,Driving Floating Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),14,50,135,1,0,Regular Season,PHI,BOS,Away,2022-10-18,2022-2023,11:05,665
3,0022200001,14,Tobias Harris,1,11,3,Made Shot,Tip Layup Shot,2PT Field Goal,Restricted Area,Center(C),0,0,0,1,1,Regular Season,PHI,BOS,Away,2022-10-18,2022-2023,11:03,663
4,0022200001,15,Jayson Tatum,1,10,46,Made Shot,Jump Shot,3PT Field Goal,Left Corner 3,Left Side(L),23,-232,49,1,1,Regular Season,BOS,PHI,Home,2022-10-18,2022-2023,10:46,646


In [121]:
nba_shot_data_final.to_csv('nba_shot_data_v4.csv', index=False)